# Spørring mot verksregisteret

Oddrun Ohren 

Lars G Johnsen 

## Kode

In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import dhlab.text as dh
import dhlab.api.dhlab_api as dhapi

In [2]:
%%HTML
<style>
p {
    font-size:1.3em;
};
h1, h2, h3, h4 {
  background-color: black;
  color: white;
}
</style>

In [3]:
def query_verksregister(forfatter="", tittel="", page = 1):
    """spør mot verksregisteret 
    api for webgrensesnittet https://livedata.bibsys.no/verksregister/"""
    
    # sett opp query
    params = {
        "query" : f"(forfatter:{forfatter}) AND (tittel:{tittel})",
        "page": page
    }
    
    #utfør spørring
    res = requests.get("https://livedata.bibsys.no/works-registry-api/search/searchWorksbyTitleAndCreator", params = params)
    if res.status_code == 200:
        res = res.text
    else:
        res = None
    
    return res

In [4]:
def find_works(forfatter="", tittel=""):
    """finn verk i registeret
    returner dem som pandas dataramme"""
    
    query_res = query_verksregister(forfatter, tittel)
    if query_res is None:
        return None
    soup = BeautifulSoup(query_res)
    n = soup.find("p")
    struct = json.loads(n.text)
    res = pd.DataFrame(struct['results'])
    return res

In [5]:
def works(identifier):
    """Hent ut data om et verk fra identifikatoren"""
    
    res = requests.get(f"https://livedata.bibsys.no/works-registry-api/biblio/byWork/{identifier}")
    try:
        r = json.loads(res.text)
    except:
        r = {}
    return r

In [6]:
def url_to_urn(url):
    """Pell ut URN fra en URL"""

    a = re.findall("URN.*", url)
    if a != []:
        res = a[0]
    else:
        res = ''
    return res

In [7]:
def urns_for_works(works_id):
    """Fra en verks-ID hent ut alle URNer
    param:
        works_id a work identifier - from """
    
    graph = works(works_id)['@graph']
    res = [[url_to_urn(x['id']) for x in i['bibsys:url_to_content']] for i in graph if 'bibsys:url_to_content' in i]
    result = [x for y in res for x in y if x != '']
    return result

# Let etter en bestemt bok

Det her kan sikkert generaliseres på en måte, men tenker at det er litt manuelt arbeid i å konstruere korpuset.

In [8]:
result = find_works("camilla collett", "amtmannens døtre")

Søket gir en dataramme, ideelt inneholder en en rad, der første rad er verksidentiteten.

In [9]:
result

,id,title,language,quality,created,manifested,creators,types,deleted
0,5f966dca3672b0948f66c148a8259562bb33807cb467ce...,Amtmandens Døttre,nob,kat3,2018-07-24T11:46:44.000+0000,1855-01-01T00:00:00.000+0000,"[{'name': 'Collett, Camilla', 'birth': 1813, '...","[{'id': 'V1000', 'labels': {'no': 'Språkbasert...",False


Finn alle URNer for verket

In [10]:
urns = urns_for_works(result.id[0])

In [11]:
urns

['URN:NBN:no-nb_digibok_2009072112001',
 'URN:NBN:no-nb_digibok_2011032520038',
 'URN:NBN:no-nb_digibok_2009072212001',
 'URN:NBN:no-nb_digibok_2009072312001',
 'URN:NBN:no-nb_digibok_2006082400051',
 'URN:NBN:no-nb_digibok_2011070620007',
 'URN:NBN:no-nb_digibok_2009071712001',
 'URN:NBN:no-nb_digibok_2012112206221',
 'URN:NBN:no-nb_digibok_2006083100087',
 'URN:NBN:no-nb_digibok_2014111008005',
 'URN:NBN:no-nb_digibok_2013042508156',
 'URN:NBN:no-nb_digibok_2009072812002',
 'URN:NBN:no-nb_digibok_2014092308143',
 'URN:NBN:no-nb_digibok_2014073108005',
 'URN:NBN:no-nb_digibok_2014092608039',
 'URN:NBN:no-nb_digibok_2010102906097',
 'URN:NBN:no-nb_digibok_2010111506037',
 'URN:NBN:no-nb_digibok_2008022001004',
 'URN:NBN:no-nb_digibok_2009072912001',
 'URN:NBN:no-nb_digibok_2008070700052']

Let opp URN-ene i dhlab, og sorter dem på år. Datarammen gir det som skal til for å konstruere parallelltekster. Det er en del titler som inneholder flere tekster, så det må sjekkes. Ellers kan tekster velges ut mer eller mindre automatisk fra årsspenn, og knyttes til språkendringsperioder som 1907, 1917, 193x, 19.. og flere.

In [12]:
dhapi.get_metadata(urns).sort_values(by = 'year')

,dhlabid,title,authors,urn,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
1,100487730,Amtmandens Døttre : en Fortælling. D. 1,"Collett , Camilla",URN:NBN:no-nb_digibok_2011032520038,oai:nb.bibsys.no:990304736244702202,fbf5c6432c86426817fe167282f603ad,,Christiania,18550101,1855,Johan Dahl,nob,,,fiction,Skjønnlitteratur,digibok
0,100270441,Amtmandens Døttre : en Fortælling,"Collett , Camilla",URN:NBN:no-nb_digibok_2009072112001,oai:nb.bibsys.no:990220947334702202,685249c68d39b323c57fb8cbe85b60b5,,Kjøbenhavn,18600101,1860,Gyldendal,nob,,,novel,Skjønnlitteratur,digibok
2,100227442,Amtmannens döttrar : en norsk berättelse,"Collett , Camilla",URN:NBN:no-nb_digibok_2009072212001,oai:nb.bibsys.no:990318480874702202,eaa23da3e9dfeaec18978407b7646115,,Stockholm,18630101,1863,Hierta,swe / nob,,,novel,Skjønnlitteratur,digibok
16,100238394,Camilla Colletts Skrifter. 3 Første Del : Amtm...,"Collett , Camilla",URN:NBN:no-nb_digibok_2009072912001,oai:nb.bibsys.no:999608459174702202,debbc94bed936a3e4b4cf745b044d119,,Kristiania,18920101,1892,Cammermeyer,nob,skjønnlitteratur,,,Uklassifisert,digibok
9,100238393,Camilla Colletts Skrifter. 4 Anden Del : Amtma...,"Collett , Camilla",URN:NBN:no-nb_digibok_2009072812002,oai:nb.bibsys.no:999100149304702202,7b6284cccbba3bee69d857c2b149ee71,,Kristiania,18930101,1893,Cammermeyer,nob,skjønnlitteratur,,novel,Skjønnlitteratur,digibok
4,100259566,Amtmandens Døtre,"Collett , Camilla",URN:NBN:no-nb_digibok_2009071712001,oai:nb.bibsys.no:997820352644702202,2c9a832c4d56b27053ffceef2ed43db2,,Kristiania,18970101,1897,Cammermeyer,nob,,,,Uklassifisert,digibok
12,100224370,Camilla Collett i utvalg,"Collett , Camilla / Agerholt , Anna Caspari",URN:NBN:no-nb_digibok_2014092608039,oai:nb.bibsys.no:999301691074702202,896ad2e94eeb0d324fe6bf3c6ee45b65,,Oslo,19300101,1930,Gyldendal,nob,,,,Uklassifisert,digibok
13,100021180,Amtmannens døtre,"Collett , Camilla",URN:NBN:no-nb_digibok_2010102906097,oai:nb.bibsys.no:999422311714702202,19818a84f58c9f8396eb45c7264ae1d5,,Oslo,19460101,1946,Gyldendal,nob,,,novel,Skjønnlitteratur,digibok
10,100226158,Amtmannens døtre,"Collett , Camilla",URN:NBN:no-nb_digibok_2014092308143,oai:nb.bibsys.no:999113756864702202,19ec2f7359a12f3680b8e295bc133e68,,Oslo,19510101,1951,Gyldendal,nob,,,novel,Skjønnlitteratur,digibok
6,100508714,Amtmannens døtre ; og I de lange Nætter,"Collett , Camilla",URN:NBN:no-nb_digibok_2006083100087,oai:nb.bibsys.no:998530102854702202,835544a34f96f4be5597d679cada151a,8205038279,Oslo,19630101,1963,Gyldendal,nob,norske / romaner,839.93,fiction,Skjønnlitteratur,digibok


In [ ]:
amtmann_korpus = dh.urnlist

In [ ]:
amtmann_korpus.corpus